![Renode](https://antmicro.com/OpenSource/assets/images/projects/renode.png)

## Intro + How to run this

This notebook shows how Renode, an open source IoT/embedded simulation framework from [Antmicro](https://antmicro.com) can be used for simulation a virtual Zephyr board with RISC-V and show a sample execution of TensorFlow Lite magic wand demo.

This runs entirely in the browser - just use **Runtime -> Run all** and see the output!

For more backround, see the [README](https://github.com/mgielda/renode-notebooks-ipynb/tree/master) of the master repository.

In [ ]:
import os

!mkdir -p renode && cd renode && curl -L https://dl.antmicro.com/projects/renode/builds/renode-latest.linux-portable.tar.gz | tar xz --strip 1
os.environ['PATH'] = os.getcwd()+"/renode:"+os.environ['PATH']

!pip3 install git+https://github.com/mgielda/pyrenode robotframework==3.1 netifaces requests psutil pyyaml

!mkdir -p litex_tflite && cd litex_tflite && curl -L https://github.com/antmicro/litex-vexriscv-tensorflow-lite-demo/archive/colab-demo.tar.gz | tar xz --strip 1

## Housekeeping

Some initialization. Import a simplistic pyrenode lib to talk to Renode over telnet, then shutdown Renode just in case it was running and create a new instance.

In [ ]:
from pyrenode import *
shutdown_renode()
connect_renode() # this sets up a log file, and clears the simulation (just in case)

## Running the demo
The demo can be run either on hardware or in the Renode simulator. 
Running on hardware requires a setup with the Digilent [Arty A7-35T](https://store.digilentinc.com/arty-a7-artix-7-fpga-development-board-for-makers-and-hobbyists/) board 
with [PmodACL](https://store.digilentinc.com/pmod-acl-3-axis-accelerometer/) connected to the JD port.

![Arty with PMOD](https://antmicro.com/blog/images/arty-with-pmod.jpg)

## Use your binary
You can build your own binary file, see the [litex-vexriscv-tensorflow-lite-demo](https://github.com/antmicro/litex-vexriscv-tensorflow-lite-demo) repository for building instructions.
Upload your binary to the hosted runtime (using the Files tab on the left) and provide path for $zephyr_elf.

tell_renode('$zephyr_elf=@path/to/elf/file')

In [ ]:
tell_renode('$zephyr_elf=@../litex_tflite/binaries/magic_wand/zephyr.elf')
tell_renode('i @../litex_tflite/renode/litex-vexriscv-tflite.resc')
print(read_until('(machine-0)').strip())

# Put the UART output to file so that we can print it easily after the run
tell_renode('uart CreateFileBackend "uart_output"')

# Also put the UART output in the log - this allows us to wait for specific lines
tell_renode('showAnalyzer uart Antmicro.Renode.Analyzers.LoggingUartAnalyzer')
tell_renode('s')
print(read_until('Starting emulation...').strip())

# The default test data is detected as a circle. The following logs are the middle and the end of the circle
expect_log("    *           *", 45)
expect_log("          *")

shutdown_renode()

In [ ]:
uart_output = open('renode/uart_output','r')
content = uart_output.read()
print(content)

Automated testing with Robot Framework!

In [ ]:
!cd renode && ./test.sh ../litex_tflite/renode/litex-vexriscv-tflite.robot

In [ ]:
from google.colab import files
files.download('renode/log.html')